In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive/')
path='/content/gdrive/My Drive/Colab Notebooks/06_Anomaly/'

import os
os.chdir(path)


In [1]:
%pip install grad-cam
%pip install timm


Defaulting to user installation because normal site-packages is not writeable
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Using cached idna-2.6-py2.py3-none-any.whl (56 kB)
  Using cached urllib3-1.22-py2.py3-none-any.whl (132 kB)
  Using cached chardet-3.0.4-py2.py3-none-any.whl (133 kB)
  Created wheel for grad-cam: filename=grad_cam-1.4.6-py3-none-any.whl size=38295 sha256=46ca4f1a82caab0043edc00620dbcb898d442308f3fd6f9ed1a18e6d2fe84f73
  Stored in directory: c:\users\jisoo\appdata\local\pip\cache\wheels\9b\ae\bd\221d138b169c4867b2ae5c893474d0b23bae6e4750bc95bc55
Successfully built grad-cam
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.12
    Uninstalling urllib3-1.26.12:
      Successf

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.1.5 requires pyqt5<5.13, which is not installed.
spyder 5.1.5 requires pyqtwebengine<5.13, which is not installed.
daal4py 2021.5.0 requires daal==2021.4.0, which is not installed.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.
anaconda-project 0.10.2 requires ruamel-yaml, which is not installed.
tensorboard 2.10.0 requires requests<3,>=2.21.0, but you have requests 2.18.4 which is incompatible.
botocore 1.23.26 requires urllib3<1.27,>=1.25.4, but 

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from pytorch_grad_cam import GradCAM
import timm
import torch
import cv2
import numpy as np

In [ ]:
device = torch.device('cuda')

In [ ]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.model = timm.create_model('efficientnet_b4', pretrained=True, num_classes=88)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
model = Network().to(device)
model.load_state_dict(torch.load("./model/best0419/0.pt"))

In [ ]:
target_layers = [model.model.blocks[-1]]
cam = GradCAM(model=model.model, target_layers=target_layers, use_cuda=True)
cam.batch_size = 2

In [ ]:
from pytorch_grad_cam.utils.image import show_cam_on_image
grayscale_cam = graycam[0, :]
rgb_img = cv2.imread("10008.png")[:, :, ::-1]
rgb_img = cv2.resize(rgb_img, (512, 512))
rgb_img = np.float32(rgb_img) / 255

cam_image=show_cam_on_image(rgb_img,grayscale_cam)
cv2.imwrite('grad_cam.jpg', cam_image)